# (01) rebuttal jobs: temp

**Motivation**: Changing ```temp_stop``` and other temperature related variables like anneal type or hard forward. using the ```vH16``` dataset. Host: ```mach```. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-vae/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-vae/figs')
tmp_dir = os.path.join(git_dir, 'jb-vae/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_PoissonVAE'))
from figures.fighelper import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Setup

In [2]:
from analysis.helper import job_runner_script


def _cleanup(path, host=None):
    for f in os.listdir(path):
        cond = f.endswith('.txt')
        if host is not None:
            cond = cond and host in f
        if cond:
            os.remove(pjoin(path, f))


def _name(host, gpu_i, fit_i):
    return f"{host}-cuda{gpu_i}-fit{fit_i}"

In [3]:
save_dir = 'Dropbox/git/_PoissonVAE/scripts'
save_dir = pjoin(os.environ['HOME'], save_dir)
os.makedirs(save_dir, exist_ok=True)

# delete existing job runners?
_cleanup(save_dir, None)

print(sorted(os.listdir(save_dir)))

['fit_vae.sh', 'kill_screens.sh', 'resume_fit.sh', 'run_sessions.sh', 'test_tqdm.py', 'test_tqdm.sh']

## Host: mach

In [4]:
host = 'mach'
assert host == os.uname().nodename
_cleanup(save_dir, host)

scripts_mach = collections.defaultdict(list)
tot = 0

### Create Fits

In [5]:
n_seeds = 3
seeds = range(1, n_seeds + 1)

t_hard = [False, True]
t_anneal = ['lin', 'exp']
t_stop = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]

archi = ['lin|lin', 'conv+b|lin']

In [6]:
combos_outer = itertools.product(
    t_hard, t_anneal)

for gpu_i, (hard_fwd, anneal_type) in enumerate(combos_outer):
    combos_inner = itertools.product(
        seeds, t_stop, archi)
    for s, t, a in combos_inner:
        # make arg
        cm = 'hard' if hard_fwd else 'soft'
        cm = f"{cm}+{anneal_type}+t{t:0.3g}"
        arg = ' '.join([
            f"--hard_fwd {str(hard_fwd).lower()}",
            f"--temp_anneal_type {anneal_type}",
            f"--temp_stop {t}",
            f"--comment {cm}",
        ])
        kws = dict(
            dataset='vH16',
            model='poisson',
            device=gpu_i,
            archi=a,
            seed=s,
            args=arg,
        )
        scripts_mach[gpu_i].append(job_runner_script(**kws))
        tot += 1

In [7]:
print(tot)

168

In [8]:
scripts_mach = dict(scripts_mach)
print({k: len(v) for k, v in scripts_mach.items()})

{0: 42, 1: 42, 2: 42, 3: 42}

### Save

In [9]:
n_fits = 6

for gpu_i, scripts in scripts_mach.items():
    scripts_divided = divide_list(scripts, n_fits)
    for fit_i, s in enumerate(scripts_divided):
        combined = ' && '.join(s)
        save_obj(
            obj=combined,
            file_name=_name(host, gpu_i, fit_i),
            save_dir=save_dir,
            mode='txt',
        )

[PROGRESS] 'mach-cuda0-fit0.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda0-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

...

[PROGRESS] 'mach-cuda3-fit1.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit2.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit3.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit4.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

[PROGRESS] 'mach-cuda3-fit5.txt' saved at
/home/hadi/Dropbox/git/_PoissonVAE/scripts

### Print one to check

In [10]:
print(combined.replace('&& ', '&& \n'))

./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.2 
--comment hard+exp+t0.2 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.3 
--comment hard+exp+t0.3 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.3 
--comment hard+exp+t0.3 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.4 
--comment hard+exp+t0.4 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.4 
--comment hard+exp+t0.4 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.5 
--comment hard+exp+t0.5 && 
./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 3 --hard_fwd true --temp_anneal_type exp --temp_stop 0.5 
--comment hard+exp+t0.5

In [11]:
print(scripts_mach)

{
    0: [
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.01 --comment soft+lin+t0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.01 --comment soft+lin+t0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.05 --comment soft+lin+t0.05",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.05 --comment soft+lin+t0.05",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.1 --comment soft+lin+t0.1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.1 --comment soft+lin+t0.1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.2 --comment soft+lin+t0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.2 --comment soft+lin+t0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.3 --comment soft+lin+t0.3",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.3 --comment soft+lin+t0.3",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.4 --comment soft+lin+t0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.4 --comment soft+lin+t0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.5 --comment soft+lin+t0.5",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.5 --comment soft+lin+t0.5",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.01 --comment soft+lin+t0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.01 --comment soft+lin+t0.01",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.05 --comment soft+lin+t0.05",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.05 --comment soft+lin+t0.05",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.1 --comment soft+lin+t0.1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.1 --comment soft+lin+t0.1",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.2 --comment soft+lin+t0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.2 --comment soft+lin+t0.2",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.3 --comment soft+lin+t0.3",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.3 --comment soft+lin+t0.3",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.4 --comment soft+lin+t0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd false --temp_anneal_type lin 
--temp_stop 0.4 --comment soft+lin+t0.4",
        "./fit_vae.sh '0' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd false --temp_anneal_type lin --temp_stop 
0.5 --comment soft+lin+t0.5",
        "./fit_vae.

In [12]:
print(scripts_divided)

[
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.01 --comment hard+exp+t0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.01 --comment hard+exp+t0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.05 --comment hard+exp+t0.05",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.05 --comment hard+exp+t0.05",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.1 --comment hard+exp+t0.1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.1 --comment hard+exp+t0.1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.2 --comment hard+exp+t0.2"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.2 --comment hard+exp+t0.2",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.3 --comment hard+exp+t0.3",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.3 --comment hard+exp+t0.3",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.4 --comment hard+exp+t0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.4 --comment hard+exp+t0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.5 --comment hard+exp+t0.5",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 1 --hard_fwd true --temp_anneal_type exp --temp_stop
0.5 --comment hard+exp+t0.5"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.01 --comment hard+exp+t0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop
0.01 --comment hard+exp+t0.01",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.05 --comment hard+exp+t0.05",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop
0.05 --comment hard+exp+t0.05",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.1 --comment hard+exp+t0.1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop
0.1 --comment hard+exp+t0.1",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.2 --comment hard+exp+t0.2"
    ],
    [
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop
0.2 --comment hard+exp+t0.2",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.3 --comment hard+exp+t0.3",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop
0.3 --comment hard+exp+t0.3",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.4 --comment hard+exp+t0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'conv+b|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop
0.4 --comment hard+exp+t0.4",
        "./fit_vae.sh '3' 'vH16' 'poisson' 'lin|lin' --seed 2 --hard_fwd true --temp_anneal_type exp --temp_stop 
0.5 --comment hard+exp+t0.5",
        "./fit_vae.sh '3' 